<a href="https://colab.research.google.com/github/cjbcdk/beer-detection/blob/main/YOLO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python yolov5/train.py --img 640 --cfg yolov5/models/yolov5s.yaml --hyp yolov5/data/hyps/hyp.scratch.yaml --batch 8 --epochs 5 --data beer_coke.yaml --weights yolov5/yolov5s.pt --workers 4 --name beer_coke_detector

^C


In [ ]:
!python yolov5\detect.py --source 0 --weights models\best.pt

detect: weights=['models\\best.pt'], source=0, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5\runs\detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5  2021-12-2 torch 1.9.0 CUDA:0 (GeForce GTX 1650, 4096MiB)

Fusing layers... 
Model Summary: 213 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
1/1: 0...  Success (inf frames 640x480 at 30.00 FPS)

0: 480x640 1 beer, Done. (0.548s)
0: 480x640 Done. (0.010s)
0: 480x640 Done. (0.010s)
0: 480x640 Done. (0.010s)
0: 480x640 Done. (0.009s)
0: 480x640 1 beer, Done. (0.010s)
0: 480x640 1 beer, Done. (0.009s)
0: 480x640 1 beer, Done. (0.011s)
0: 480x640 1 beer, Done. (0.010s)
0: 480x640 1 beer, Done. (0.010s)
0: 480x640 1 beer, Done. (0.010s)
0: 480x640 1 beer, Done. (0.010s)
0: 

0: 480x640 Done. (0.011s)
0: 480x640 1 beer, Done. (0.012s)
0: 480x640 Done. (0.011s)
0: 480x640 1 beer, Done. (0.012s)
0: 480x640 1 beer, Done. (0.012s)
0: 480x640 1 beer, Done. (0.013s)
0: 480x640 Done. (0.011s)
0: 480x640 Done. (0.013s)
0: 480x640 1 beer, Done. (0.012s)
0: 480x640 Done. (0.012s)
0: 480x640 1 beer, Done. (0.012s)
0: 480x640 1 beer, Done. (0.012s)
0: 480x640 Done. (0.012s)
0: 480x640 Done. (0.011s)
0: 480x640 Done. (0.012s)
0: 480x640 Done. (0.013s)
0: 480x640 Done. (0.011s)
0: 480x640 Done. (0.012s)
0: 480x640 Done. (0.012s)
0: 480x640 Done. (0.013s)
0: 480x640 Done. (0.011s)
0: 480x640 Done. (0.012s)
0: 480x640 Done. (0.011s)
0: 480x640 Done. (0.012s)
0: 480x640 Done. (0.011s)
0: 480x640 Done. (0.011s)
0: 480x640 Done. (0.012s)
0: 480x640 Done. (0.013s)
0: 480x640 Done. (0.011s)
0: 480x640 Done. (0.013s)
0: 480x640 Done. (0.012s)
0: 480x640 Done. (0.013s)
0: 480x640 Done. (0.011s)
0: 480x640 Done. (0.013s)
0: 480x640 Done. (0.011s)
0: 480x640 Done. (0.011s)
0: 480x6

In [ ]:
import torch
from IPython.display import Image  # for displaying images
import os 
import random
import shutil
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
from xml.dom import minidom
from tqdm import tqdm
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from scipy.spatial import distance as dist
from collections import OrderedDict
import numpy as np

class CentroidTracker():
	def __init__(self, maxDisappeared=10):
		# initialize the next unique object ID along with two ordered
		# dictionaries used to keep track of mapping a given object
		# ID to its centroid and number of consecutive frames it has
		# been marked as "disappeared", respectively
		self.nextObjectID = 0
		self.objects = OrderedDict()
		self.disappeared = OrderedDict()

		# store the number of maximum consecutive frames a given
		# object is allowed to be marked as "disappeared" until we
		# need to deregister the object from tracking
		self.maxDisappeared = maxDisappeared

	def register(self, centroid):
		# when registering an object we use the next available object
		# ID to store the centroid
		self.objects[self.nextObjectID] = centroid
		self.disappeared[self.nextObjectID] = 0
		self.nextObjectID += 1

	def deregister(self, objectID):
		# to deregister an object ID we delete the object ID from
		# both of our respective dictionaries
		del self.objects[objectID]
		del self.disappeared[objectID]

	def update(self, rects):
		# check to see if the list of input bounding box rectangles
		# is empty
		if len(rects) == 0:
			# loop over any existing tracked objects and mark them
			# as disappeared
			for objectID in list(self.disappeared.keys()):
				self.disappeared[objectID] += 1

				# if we have reached a maximum number of consecutive
				# frames where a given object has been marked as
				# missing, deregister it
				if self.disappeared[objectID] > self.maxDisappeared:
					self.deregister(objectID)

			# return early as there are no centroids or tracking info
			# to update
			return self.objects

		# initialize an array of input centroids for the current frame
		inputCentroids = np.zeros((len(rects), 2), dtype="int")

		# loop over the bounding box rectangles
		for (i, (startX, startY, endX, endY)) in enumerate(rects):
			# use the bounding box coordinates to derive the centroid
			cX = int((startX + endX) / 2.0)
			cY = int((startY + endY) / 2.0)
			inputCentroids[i] = (cX, cY)

		# if we are currently not tracking any objects take the input
		# centroids and register each of them
		if len(self.objects) == 0:
			for i in range(0, len(inputCentroids)):
				self.register(inputCentroids[i])

		# otherwise, are are currently tracking objects so we need to
		# try to match the input centroids to existing object
		# centroids
		else:
			# grab the set of object IDs and corresponding centroids
			objectIDs = list(self.objects.keys())
			objectCentroids = list(self.objects.values())

			# compute the distance between each pair of object
			# centroids and input centroids, respectively -- our
			# goal will be to match an input centroid to an existing
			# object centroid
			D = dist.cdist(np.array(objectCentroids), inputCentroids)

			# in order to perform this matching we must (1) find the
			# smallest value in each row and then (2) sort the row
			# indexes based on their minimum values so that the row
			# with the smallest value as at the *front* of the index
			# list
			rows = D.min(axis=1).argsort()

			# next, we perform a similar process on the columns by
			# finding the smallest value in each column and then
			# sorting using the previously computed row index list
			cols = D.argmin(axis=1)[rows]

			# in order to determine if we need to update, register,
			# or deregister an object we need to keep track of which
			# of the rows and column indexes we have already examined
			usedRows = set()
			usedCols = set()

			# loop over the combination of the (row, column) index
			# tuples
			for (row, col) in zip(rows, cols):
				# if we have already examined either the row or
				# column value before, ignore it
				# val
				if row in usedRows or col in usedCols:
					continue

				# otherwise, grab the object ID for the current row,
				# set its new centroid, and reset the disappeared
				# counter
				objectID = objectIDs[row]
				self.objects[objectID] = inputCentroids[col]
				self.disappeared[objectID] = 0

				# indicate that we have examined each of the row and
				# column indexes, respectively
				usedRows.add(row)
				usedCols.add(col)

			# compute both the row and column index we have NOT yet
			# examined
			unusedRows = set(range(0, D.shape[0])).difference(usedRows)
			unusedCols = set(range(0, D.shape[1])).difference(usedCols)

			# in the event that the number of object centroids is
			# equal or greater than the number of input centroids
			# we need to check and see if some of these objects have
			# potentially disappeared
			if D.shape[0] >= D.shape[1]:
				# loop over the unused row indexes
				for row in unusedRows:
					# grab the object ID for the corresponding row
					# index and increment the disappeared counter
					objectID = objectIDs[row]
					self.disappeared[objectID] += 1

					# check to see if the number of consecutive
					# frames the object has been marked "disappeared"
					# for warrants deregistering the object
					if self.disappeared[objectID] > self.maxDisappeared:
						self.deregister(objectID)

			# otherwise, if the number of input centroids is greater
			# than the number of existing object centroids we need to
			# register each new input centroid as a trackable object
			else:
				for col in unusedCols:
					self.register(inputCentroids[col])

		# return the set of trackable objects
		return self.objects

In [ ]:
model = torch.hub.load('yolov5', 'custom', path='models/yolo_model3.pt', source='local')

# Images
dir = 'yolo_data/images/test/frame_001599.jpg'
imgs = [dir]  # batch of images

# Inference
results = model(imgs)
# or .show(), .save()

print(results.xyxy)

YOLOv5  v6.0-119-g00e308f torch 1.9.0 CUDA:0 (GeForce GTX 1650, 4096MiB)

Fusing layers... 
Model Summary: 213 layers, 7015519 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


[tensor([[405.75000,   0.62500, 464.25000,  42.56250,   0.94141,   0.00000],
        [450.00000, 326.75000, 544.00000, 369.75000,   0.93262,   0.00000],
        [140.75000, 319.75000, 229.25000, 364.75000,   0.92139,   0.00000],
        [311.75000, 158.00000, 363.25000, 244.25000,   0.89893,   1.00000],
        [355.25000, 164.00000, 409.25000, 248.00000,   0.89111,   1.00000]], device='cuda:0')]


In [ ]:
from imutils.video import VideoStream
import numpy as np
import argparse
import imutils
import time
import cv2
import os
import torch
from PIL import Image
import xml.etree.ElementTree as ET
#from google.colab.patches import cv2_imshow


class Take_test_image(torch.utils.data.Dataset):
    def __init__(self, root, transforms=None):
        self.root = root
        self.transforms = transforms
        self.imgs = list(sorted(os.listdir(os.path.join(self.root,"images_test1"))))

    def __getitem__(self, idx):
        img_path = os.path.join(self.root, "images_test", self.imgs[idx])
        img = Image.open(img_path).convert("RGB")
        target={}
        
        if self.transforms is not None:
            img, target = self.transforms(img, target)
        return img, target
        
    def __len__(self):
        return len(self.imgs)


img_array = []
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

ct_beer = CentroidTracker()
ct_coke = CentroidTracker()
#picture = Take_test_image(drive_path, get_transform(train=False))
num_images=10000
model.to(device)
model.eval()
global_label=[]
cap = cv2.VideoCapture("video_vero.mp4")
ret=1
i =0
while i in range(ret+1):

  #path=os.path.join(drive_path, 'images_test1', 'frame_'+str(i).zfill(6)+'.jpg')
  ret, frame = cap.read()

  frame1 = frame
  cv2.imwrite('image.jpg', frame) 
  tic = time.perf_counter()
  torch.cuda.synchronize()
  detection = model("image.jpg")
  detection=detection.xyxy
  #cv2.imshow("image",frame)
  #print(detection)

  box1=detection[0][:,0:4]
 # print(box1)
  scores = detection[0][:,4]
 # print(scores)
  labels=detection[0][:,5]
 # print(labels)
  labels=list(labels)
  scores=list(scores)
  labels1=labels
  box1=list(box1)
  rects_beer=[]
  rects_coke=[]
  m=0
  centerx=[]
	# loop over the detections
  for j in range(len(scores)):
    # filter out weak detections by ensuring the predicted
    # probability is greater than a minimum threshold
    print(scores[j])
    if scores[j] > 0.78:
      # compute the (x, y)-coordinates of the bounding box for
			# the object, then update the bounding box rectangles list
      
      box = np.array(box1[j].cpu())     
			# draw a bounding box surrounding the object so we can
			# visualize it
      (startX, startY, endX, endY) = box.astype("int")
      if labels[j]==0:
        rects_beer.append(box.astype("int"))
        cv2.rectangle(frame1, (startX, startY), (endX, endY),(0, 255, 0), 2)
      if labels[j]==1:
        rects_coke.append(box.astype("int"))
        cv2.rectangle(frame1, (startX, startY), (endX, endY),(0, 0, 255), 2)
	# update our centroid tracker using the computed set of bounding
	# box rectangles
  objects_beer = ct_beer.update(rects_beer)
  objects_coke = ct_coke.update(rects_coke)
	# loop over the tracked objects
  for (objectID, centroid) in objects_beer.items():    
   	# draw both the ID of the object and the centroid of the
		# object on the output frame
    text = "Beer {}".format(objectID)
    cv2.putText(frame1, text, (centroid[0] - 10, centroid[1] - 10),
    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    cv2.circle(frame1, (centroid[0], centroid[1]), 4, (0, 255, 0), -1)
  for (objectID, centroid) in objects_coke.items():    
   	# draw both the ID of the object and the centroid of the
		# object on the output frame
    text = "Coke {}".format(objectID)
    cv2.putText(frame1, text, (centroid[0] - 10, centroid[1] - 10),
    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
    cv2.circle(frame1, (centroid[0], centroid[1]), 4, (0, 0, 255), -1)
  toc = time.perf_counter()
  torch.cuda.synchronize()
  elapsed_fp = toc-tic
  print(elapsed_fp)



	# show the output frame
  #clear_output()
  cv2.imshow("image",frame1)
  key = cv2.waitKey(1) & 0xFF
  img_array.append(frame1)

	# if the `q` key was pressed, break from the loop
  if key == ord("q"):
   break
height, width, layers = frame.shape
size = (width,height)
out = cv2.VideoWriter('video_traking2.avi',cv2.VideoWriter_fourcc(*'DIVX'), 30, size)
i=0
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()

tensor(0.93311, device='cuda:0')
tensor(0.92822, device='cuda:0')
tensor(0.92432, device='cuda:0')
tensor(0.87305, device='cuda:0')
0.0757860000000079
tensor(0.93506, device='cuda:0')
tensor(0.92969, device='cuda:0')
tensor(0.92725, device='cuda:0')
tensor(0.87012, device='cuda:0')
0.07724660000000938
tensor(0.93457, device='cuda:0')
tensor(0.92969, device='cuda:0')
tensor(0.92578, device='cuda:0')
tensor(0.86816, device='cuda:0')
0.07127129999999227
tensor(0.93457, device='cuda:0')
tensor(0.92969, device='cuda:0')
tensor(0.91943, device='cuda:0')
tensor(0.86816, device='cuda:0')
0.0652106999999944
tensor(0.93408, device='cuda:0')
tensor(0.92871, device='cuda:0')
tensor(0.92285, device='cuda:0')
tensor(0.87012, device='cuda:0')
0.059510399999993524
tensor(0.93115, device='cuda:0')
tensor(0.92920, device='cuda:0')
tensor(0.92188, device='cuda:0')
tensor(0.85938, device='cuda:0')
0.056415599999994015
tensor(0.93213, device='cuda:0')
tensor(0.93115, device='cuda:0')
tensor(0.93018, device

tensor(0.93555, device='cuda:0')
tensor(0.93359, device='cuda:0')
tensor(0.92578, device='cuda:0')
tensor(0.92188, device='cuda:0')
tensor(0.86963, device='cuda:0')
0.06269660000000954
tensor(0.94141, device='cuda:0')
tensor(0.93262, device='cuda:0')
tensor(0.92139, device='cuda:0')
tensor(0.89893, device='cuda:0')
tensor(0.89111, device='cuda:0')
0.0632331999999991
tensor(0.94043, device='cuda:0')
tensor(0.93408, device='cuda:0')
tensor(0.92334, device='cuda:0')
tensor(0.88086, device='cuda:0')
tensor(0.86963, device='cuda:0')
0.06446549999999718
tensor(0.94580, device='cuda:0')
tensor(0.93311, device='cuda:0')
tensor(0.92090, device='cuda:0')
tensor(0.88428, device='cuda:0')
tensor(0.87988, device='cuda:0')
0.06083420000000217
tensor(0.94824, device='cuda:0')
tensor(0.93115, device='cuda:0')
tensor(0.92236, device='cuda:0')
tensor(0.86426, device='cuda:0')
tensor(0.76709, device='cuda:0')
0.06110510000000602
tensor(0.94824, device='cuda:0')
tensor(0.93164, device='cuda:0')
tensor(0.9

tensor(0.94385, device='cuda:0')
tensor(0.93506, device='cuda:0')
tensor(0.93311, device='cuda:0')
0.062466499999999314
tensor(0.94385, device='cuda:0')
tensor(0.93750, device='cuda:0')
tensor(0.93213, device='cuda:0')
0.06011069999999563
tensor(0.94434, device='cuda:0')
tensor(0.93945, device='cuda:0')
tensor(0.93262, device='cuda:0')
0.05710390000000132
tensor(0.94287, device='cuda:0')
tensor(0.93848, device='cuda:0')
tensor(0.93311, device='cuda:0')
tensor(0.70898, device='cuda:0')
0.06062799999999413
tensor(0.94482, device='cuda:0')
tensor(0.93848, device='cuda:0')
tensor(0.93213, device='cuda:0')
tensor(0.83350, device='cuda:0')
0.07081010000000276
tensor(0.94482, device='cuda:0')
tensor(0.93555, device='cuda:0')
tensor(0.93408, device='cuda:0')
tensor(0.88428, device='cuda:0')
0.06560389999999927
tensor(0.94434, device='cuda:0')
tensor(0.93701, device='cuda:0')
tensor(0.93359, device='cuda:0')
tensor(0.89502, device='cuda:0')
0.06645029999999963
tensor(0.94287, device='cuda:0')
t

tensor(0.94238, device='cuda:0')
tensor(0.93262, device='cuda:0')
tensor(0.91797, device='cuda:0')
0.06462460000000192
tensor(0.93994, device='cuda:0')
tensor(0.93408, device='cuda:0')
tensor(0.90039, device='cuda:0')
0.05694799999999134
tensor(0.94043, device='cuda:0')
tensor(0.93408, device='cuda:0')
tensor(0.91357, device='cuda:0')
0.05956079999999986
tensor(0.94092, device='cuda:0')
tensor(0.93555, device='cuda:0')
tensor(0.92529, device='cuda:0')
0.06469540000000507
tensor(0.94189, device='cuda:0')
tensor(0.93359, device='cuda:0')
tensor(0.93164, device='cuda:0')
0.05680079999999066
tensor(0.94092, device='cuda:0')
tensor(0.93311, device='cuda:0')
tensor(0.90723, device='cuda:0')
0.05857709999999372
tensor(0.94043, device='cuda:0')
tensor(0.93213, device='cuda:0')
tensor(0.92090, device='cuda:0')
0.06446309999999755
tensor(0.94043, device='cuda:0')
tensor(0.93359, device='cuda:0')
tensor(0.90869, device='cuda:0')
0.06885150000000806
tensor(0.94092, device='cuda:0')
tensor(0.93311,

tensor(0.94189, device='cuda:0')
tensor(0.93164, device='cuda:0')
tensor(0.92822, device='cuda:0')
tensor(0.70361, device='cuda:0')
0.06453470000001005
tensor(0.94189, device='cuda:0')
tensor(0.93311, device='cuda:0')
tensor(0.79688, device='cuda:0')
tensor(0.45093, device='cuda:0')
0.06499589999999955
tensor(0.94043, device='cuda:0')
tensor(0.93457, device='cuda:0')
tensor(0.79297, device='cuda:0')
tensor(0.78711, device='cuda:0')
tensor(0.42261, device='cuda:0')
tensor(0.30396, device='cuda:0')
0.06781559999998876
tensor(0.94238, device='cuda:0')
tensor(0.93457, device='cuda:0')
tensor(0.87842, device='cuda:0')
tensor(0.87305, device='cuda:0')
tensor(0.80859, device='cuda:0')
0.06698260000000289
tensor(0.94189, device='cuda:0')
tensor(0.93408, device='cuda:0')
tensor(0.86621, device='cuda:0')
tensor(0.85986, device='cuda:0')
tensor(0.84814, device='cuda:0')
0.06690430000000447
tensor(0.93896, device='cuda:0')
tensor(0.93555, device='cuda:0')
tensor(0.92139, device='cuda:0')
tensor(0.

tensor(0.95264, device='cuda:0')
tensor(0.93457, device='cuda:0')
tensor(0.93213, device='cuda:0')
tensor(0.90332, device='cuda:0')
tensor(0.82812, device='cuda:0')
0.06350589999999556
tensor(0.95068, device='cuda:0')
tensor(0.93994, device='cuda:0')
tensor(0.92822, device='cuda:0')
tensor(0.92627, device='cuda:0')
tensor(0.83350, device='cuda:0')
0.06951089999999738
tensor(0.95068, device='cuda:0')
tensor(0.94043, device='cuda:0')
tensor(0.92969, device='cuda:0')
tensor(0.89746, device='cuda:0')
tensor(0.81934, device='cuda:0')
0.06613310000000183
tensor(0.95117, device='cuda:0')
tensor(0.94043, device='cuda:0')
tensor(0.92432, device='cuda:0')
tensor(0.87500, device='cuda:0')
tensor(0.84229, device='cuda:0')
0.06402359999999874
tensor(0.95361, device='cuda:0')
tensor(0.93994, device='cuda:0')
tensor(0.93115, device='cuda:0')
tensor(0.90625, device='cuda:0')
tensor(0.85742, device='cuda:0')
0.06270029999998883
tensor(0.94873, device='cuda:0')
tensor(0.93848, device='cuda:0')
tensor(0.

tensor(0.95264, device='cuda:0')
tensor(0.94629, device='cuda:0')
tensor(0.93066, device='cuda:0')
tensor(0.88086, device='cuda:0')
tensor(0.79102, device='cuda:0')
0.0689214000000078
tensor(0.95264, device='cuda:0')
tensor(0.94727, device='cuda:0')
tensor(0.93213, device='cuda:0')
tensor(0.87988, device='cuda:0')
tensor(0.81201, device='cuda:0')
0.0640507000000099
tensor(0.95361, device='cuda:0')
tensor(0.94727, device='cuda:0')
tensor(0.93115, device='cuda:0')
tensor(0.88184, device='cuda:0')
tensor(0.76172, device='cuda:0')
0.060768199999998274
tensor(0.95264, device='cuda:0')
tensor(0.94775, device='cuda:0')
tensor(0.93115, device='cuda:0')
tensor(0.87939, device='cuda:0')
tensor(0.62012, device='cuda:0')
0.06218129999999178
tensor(0.95166, device='cuda:0')
tensor(0.94873, device='cuda:0')
tensor(0.93164, device='cuda:0')
tensor(0.88086, device='cuda:0')
0.05932529999999758
tensor(0.95264, device='cuda:0')
tensor(0.94873, device='cuda:0')
tensor(0.93213, device='cuda:0')
tensor(0.8

error: OpenCV(4.5.4-dev) D:\a\opencv-python\opencv-python\opencv\modules\imgcodecs\src\loadsave.cpp:799: error: (-215:Assertion failed) !_img.empty() in function 'cv::imwrite'


In [ ]:
img_array


[array([[[ 49,  72,  93],
         [ 48,  71,  92],
         [ 51,  76,  97],
         ...,
         [ 37,  59,  74],
         [ 39,  60,  78],
         [ 39,  60,  78]],
 
        [[ 54,  77,  98],
         [ 46,  69,  90],
         [ 43,  68,  89],
         ...,
         [ 37,  59,  74],
         [ 39,  60,  78],
         [ 39,  60,  78]],
 
        [[ 54,  79, 100],
         [ 42,  67,  88],
         [ 37,  64,  85],
         ...,
         [ 39,  61,  76],
         [ 40,  61,  79],
         [ 40,  61,  79]],
 
        ...,
 
        [[ 17,  22,  22],
         [ 17,  22,  22],
         [ 17,  22,  22],
         ...,
         [ 13,  16,  16],
         [ 13,  16,  16],
         [ 13,  16,  16]],
 
        [[ 18,  23,  23],
         [ 18,  23,  23],
         [ 18,  23,  23],
         ...,
         [ 13,  16,  16],
         [ 13,  16,  16],
         [ 13,  16,  16]],
 
        [[ 18,  23,  23],
         [ 18,  23,  23],
         [ 18,  23,  23],
         ...,
         [ 13,  16,  16],
  